### Create CSVs with all results for each one of the 3 types of models (Simple CNN, Base Model and ResNet-20) 

In [1]:
import pandas as pd

In [2]:
results_cifar10 = pd.read_csv('results_cifar10.csv')
results_cifar10 = results_cifar10[results_cifar10['clf'] == 'lr']

In [3]:
results_cifar100_coarse = pd.read_csv('results_cifar100_coarse.csv')
results_cifar100_fine = pd.read_csv('results_cifar100_fine.csv')
results_fashion_mnist = pd.read_csv('results_fashion_mnist.csv')

In [4]:
all_results = results_cifar10.copy()
all_results = all_results.append(results_cifar100_coarse, ignore_index=True)
all_results = all_results.append(results_cifar100_fine, ignore_index=True)
all_results = all_results.append(results_fashion_mnist, ignore_index=True)

In [5]:
all_results['full_ds_name'] =\
        all_results['dataset'].map(str) + '_' +\
        all_results['color_space'].map(str) + '_' +\
        all_results['n_colors'].map(str)

In [6]:
all_results = all_results[['model', 'full_ds_name', 'test_acc']]

In [7]:
all_results['model'] = all_results['model'].str.replace('_cifar10_',' ')
all_results['model'] = all_results['model'].str.replace('.h5','')

all_results['model'] = all_results['model'].str.replace('simple','Simple CNN')
all_results['model'] = all_results['model'].str.replace('all_conv','Base Model CNN')
all_results['model'] = all_results['model'].str.replace('resnet20','ResNet-20')

all_results['model'] = all_results['model'].str.replace('rgb256','rgb')

In [8]:
base_models = ['Simple CNN', 'Base Model CNN', 'ResNet-20']
ds_names = sorted(all_results['full_ds_name'].unique())

dict_out = {}

for bm in base_models:
    results_base_model = all_results[all_results['model'].str.contains(bm)]
    model_names = sorted(results_base_model['model'].unique())
    
    dict_aux = {}
    for m in model_names:
        results_model = results_base_model[results_base_model['model'].str.contains(m)]
        dict_aux[m] = []
        for ds in ds_names:
            aux = ds
            if ('rgb' in ds) and ('rgb' not in m):
                aux = ds.replace('rgb', 'gray')
            
            dict_aux[m].append(results_model[results_model['full_ds_name'] == aux]['test_acc'].values[0])
            
    df_aux = pd.DataFrame.from_dict(dict_aux)
    df_aux['dataset'] = ds_names
    df_aux = df_aux[['dataset'] + model_names]
    dict_out[bm] = df_aux.copy()

In [9]:
for m in dict_out:
    aux = 'resnet20'
    if m == 'Base Model CNN':
        aux = 'base_model'
    elif m == 'Simple CNN':
        aux = 'simple'
    
    dict_out[m].to_csv('results_%s.csv' % (aux), index=False)